MSBA 502: Final Project (Part 1: Data Validation, Cleaning, Feature Engineering)

Lead: Alex Snyder

Contribution Scope (Alex): Data Validation, Cleaning, Feature Engineering, Export df_use.csv, Data Dictionary

BLOCK 0 - INTRODUCTION

In [1]:
print("Owner: Alex Snyder — Part 1: Data Validation, Cleaning, Feature Engineering")

Owner: Alex Snyder — Part 1: Data Validation, Cleaning, Feature Engineering


BLOCK 1 - IMPORTS AND DATA

In [2]:
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)
pd.set_option("display.colheader_justify", "left")

PATH = "yelp.cleaned.csv"  # exact file name you showed
df = pd.read_csv(PATH)

print(f"Rows, Cols: {df.shape}")
print("\nSample rows:")
display(df.head(3))

print("\nCompact schema:")
schema = (
    pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes.astype(str).values,
        "non_null_frac": (1 - df.isna().mean()).round(3).values
    })
    .sort_values("column")
    .reset_index(drop=True)
)
display(schema)

Rows, Cols: (15000, 20)

Sample rows:


,review_id,user_id,business_id,business_star_rating,review_date,review_text,business_name,business_city,business_state,user_star_rating,business_review_count,business_is_open,business_categories,user_name,user_review_count,user_average_stars,sentiment_label,sentiment_confidence,emotion_label,emotion_confidence
0,4zopEEPqfwm-c_FNpeHZYw,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5.0,2016-05-31 02:14:54,We were a bit weary about trying the Shellfish...,Santa Barbara Shellfish Company,Santa Barbara,CA,4.0,2404,1,"Live/Raw Food, Restaurants, Seafood, Beer Bar,...",David,338,4.12,1,0.996471,neutral,0.680843
1,LLzom-2TITa4gasV7_fCCA,jrOeN_F9lz5415C8JLUKNg,XTQ3iEaLMUbJg2DrcodslA,5.0,2017-08-07 19:51:10,Great experience purchasing a washer and dryer...,Sears Home Appliance Showroom,Springfield,PA,3.0,8,0,"Home Services, Home & Garden, Building Supplie...",Amanda,1,3.00,1,0.999464,joy,0.973149
2,CEcndvfWE_kA6gPtJOZRYw,tcT7C-lixeyg19jbuAR0jw,UYxGgnJ4Z2lOehK23HSIlw,5.0,2017-01-27 20:05:51,Awesome detailing job. My car was DIRTY. Hadn'...,Mister Car Wash,Tampa,FL,2.5,133,1,"Auto Detailing, Car Wash, Automotive",Lee,4,3.75,1,0.996508,surprise,0.473677



Compact schema:


,column,dtype,non_null_frac
0,business_categories,object,1.0
1,business_city,object,1.0
2,business_id,object,1.0
3,business_is_open,int64,1.0
4,business_name,object,1.0
5,business_review_count,int64,1.0
6,business_star_rating,float64,1.0
7,business_state,object,1.0
8,emotion_confidence,float64,1.0
9,emotion_label,object,1.0


BLOCK 3 - 

In [3]:
# --- Canonical mapping to project names ---
col_map_candidates = {
    "Stars": ["stars","star","rating","stars_rating","star_rating","user_star_rating"],
    "Sentiment": ["sentiment","polarity","computed_sentiment","sentiment_score","sentiment_label"],  # may be label today
    "Confidence": ["confidence","sentiment_confidence","sent_conf","prob","confidence_score"],
    "Emotion": ["emotion","primary_emotion","emo_label","emotion_label"],
    "Emotion_Confidence": ["emotion_confidence","emo_confidence","emotion_prob"],  # <- NEW
    "Date": ["date","review_date","created_at","time"],
    "Review": ["review","text","review_text","content","body"],

    "City": ["city","business_city"],
    "State": ["state","business_state"],
    "Category": ["category","categories","business_category","business_categories"],
    "Business_Avg_Rating": ["business_avg_rating","biz_avg","avg_rating_business","business_star_rating"],
    "User_Avg_Star_Rating": ["user_avg_star_rating","user_avg","user_mean_stars","user_average_stars"],
    "Business_Is_Open": ["is_open","business_is_open"]
}

def auto_map_columns(df, col_map_candidates):
    inv = {}
    for canonical, variants in col_map_candidates.items():
        for v in variants + [canonical]:
            inv[v.lower()] = canonical
    new_cols = {}
    for c in df.columns:
        c_low = c.strip().lower()
        new_cols[c] = inv.get(c_low, c)
    return df.rename(columns=new_cols)

df = auto_map_columns(df, col_map_candidates)

# Keep a preserved copy of the raw sentiment label (string or numeric) if it existed pre-map
if "Sentiment" in df.columns:
    df["Sentiment_Label"] = df["Sentiment"].copy()

# Convert Sentiment to numeric polarity if it’s text (−1, 0, +1)
if df["Sentiment"].dtype == "O":
    lab = df["Sentiment"].astype(str).str.strip().str.lower()
    sent_map = {"positive": 1.0, "pos": 1.0,
                "negative": -1.0, "neg": -1.0,
                "neutral": 0.0, "neu": 0.0}
    df["Sentiment"] = lab.map(sent_map).fillna(0.0)

required = ["Stars","Sentiment","Confidence","Emotion","Date","Review"]
missing = [c for c in required if c not in df.columns]
print("Missing required (post-map):", missing)
if missing:
    raise ValueError(f"Missing required columns after mapping: {missing}")

display(df.head(3))

Missing required (post-map): []


,review_id,user_id,business_id,Business_Avg_Rating,Date,Review,business_name,City,State,Stars,business_review_count,Business_Is_Open,Category,user_name,user_review_count,User_Avg_Star_Rating,Sentiment,Confidence,Emotion,Emotion_Confidence,Sentiment_Label
0,4zopEEPqfwm-c_FNpeHZYw,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5.0,2016-05-31 02:14:54,We were a bit weary about trying the Shellfish...,Santa Barbara Shellfish Company,Santa Barbara,CA,4.0,2404,1,"Live/Raw Food, Restaurants, Seafood, Beer Bar,...",David,338,4.12,1,0.996471,neutral,0.680843,1
1,LLzom-2TITa4gasV7_fCCA,jrOeN_F9lz5415C8JLUKNg,XTQ3iEaLMUbJg2DrcodslA,5.0,2017-08-07 19:51:10,Great experience purchasing a washer and dryer...,Sears Home Appliance Showroom,Springfield,PA,3.0,8,0,"Home Services, Home & Garden, Building Supplie...",Amanda,1,3.00,1,0.999464,joy,0.973149,1
2,CEcndvfWE_kA6gPtJOZRYw,tcT7C-lixeyg19jbuAR0jw,UYxGgnJ4Z2lOehK23HSIlw,5.0,2017-01-27 20:05:51,Awesome detailing job. My car was DIRTY. Hadn'...,Mister Car Wash,Tampa,FL,2.5,133,1,"Auto Detailing, Car Wash, Automotive",Lee,4,3.75,1,0.996508,surprise,0.473677,1


BLOCK 4 - TYPES & BASIC CLEANING

In [4]:
numeric_cols = [
    "Stars", "Sentiment", "Confidence",
    "Business_Avg_Rating", "User_Avg_Star_Rating",
    "user_review_count", "business_review_count",
    "Emotion_Confidence", "Sentiment_Label"
]
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

#Dates
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

#Text
df["Review"] = df["Review"].astype(str).str.strip()

# Optional binary
if "Business_Is_Open" in df.columns:
    df["Business_Is_Open"] = pd.to_numeric(df["Business_Is_Open"], errors="coerce").fillna(0).astype(int)

#Quick null check on required
required = ["Stars","Sentiment","Confidence","Emotion","Date","Review"]
null_report = df[required].isna().mean().round(4).rename("null_frac")
display(null_report)

#Drop rows missing core fields
before = len(df)
df = df.dropna(subset=["Review","Stars"])
after = len(df)
print(f"Dropped {before-after} rows lacking core fields.")

#Clamp ranges
df["Stars"] = df["Stars"].clip(lower=1, upper=5)
df["Confidence"] = df["Confidence"].clip(lower=0, upper=1)
df["Sentiment"] = df["Sentiment"].clip(lower=-1, upper=1)
if "Emotion_Confidence" in df.columns:
    df["Emotion_Confidence"] = df["Emotion_Confidence"].clip(0, 1)

Stars         0.0
Sentiment     0.0
Confidence    0.0
Emotion       0.0
Date          0.0
Review        0.0
Name: null_frac, dtype: float64

Dropped 0 rows lacking core fields.


BLOCK 5 - TEXTUAL BEHAVIORS (EXPLICIT FOR RQ1)

In [5]:
import re
token_pattern = re.compile(r"\b\w+\b")

def word_count(s): return len(token_pattern.findall(s)) if isinstance(s, str) else 0
def frac_exclaim(s):
    if not isinstance(s, str) or not s: return 0.0
    return s.count("!")/max(len(s),1)
def frac_allcaps(s):
    if not isinstance(s, str): return 0.0
    toks = token_pattern.findall(s)
    if not toks: return 0.0
    caps = sum(1 for t in toks if t.isalpha() and t.isupper() and len(t) >= 2)
    return caps/len(toks)
def has_negation(s):
    if not isinstance(s, str): return 0
    return int(bool(re.search(r"\b(no|not|never|none|n\'t|don\'t|can\'t|won\'t)\b", s, flags=re.I)))

df["Review_Len_words"] = df["Review"].map(word_count)
df["Review_Len_chars"] = df["Review"].astype(str).str.len().fillna(0)
df["ExclaimFrac"]      = df["Review"].map(frac_exclaim)
df["AllCapsFrac"]      = df["Review"].map(frac_allcaps)
df["NegationFlag"]     = df["Review"].map(has_negation)

BLOCK 6 - FEATURE NEGINEERING (SENTSCORE & BUCKETS)

In [6]:
# ----- BLOCK 6: Feature Engineering (FINAL REVISION) -----
import ast
import numpy as np
import pandas as pd

# 1) Star bucket for downstream EDA/modeling
def star_bucket(x):
    try:
        x = int(round(x))
    except Exception:
        return np.nan
    if x <= 2: return "Low"
    if x == 3: return "Mid"
    return "High"

df["Star_Bucket"] = df["Stars"].map(star_bucket)

# 2) Team-standard sentiment feature
def label_to_num_for_lily(series):
    if pd.api.types.is_numeric_dtype(series.dtype):
        vals = series.dropna().unique()
        if len(vals) and np.min(vals) >= 0 and np.max(vals) <= 1:
            return (2 * series - 1)  # convert 0/1 to -1/+1
        return series
    lab = series.astype(str).str.strip().str.lower()
    return lab.map({
        "positive": 1.0, "pos": 1.0,
        "negative": -1.0, "neg": -1.0,
        "neutral": 0.0, "neu": 0.0
    })

if "Sentiment_Label" in df.columns:
    _lbl = label_to_num_for_lily(df["Sentiment_Label"])
else:
    _lbl = df["Sentiment"]

df["computed_sentiment"] = (_lbl * df["Confidence"]).clip(-1, 1)

# 3) Weighted emotion dummies
emo_dummies = pd.get_dummies(df["Emotion"], prefix="emotion")
if "Emotion_Confidence" in df.columns:
    emo_dummies = emo_dummies.mul(df["Emotion_Confidence"], axis=0)
df = pd.concat([df, emo_dummies], axis=1)

# 4) Convenience alias
df["Review_Len"] = df["Review_Len_words"]

# 5) Primary business category (first label only)
def extract_primary_category(x):
    if pd.isna(x): 
        return np.nan
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list, tuple)) and parsed:
                return str(parsed[0]).strip()
        except Exception:
            pass
    if "," in s:
        return s.split(",")[0].strip()
    return s

if "business_categories" in df.columns:
    df["business_category1"] = df["business_categories"].apply(extract_primary_category)
elif "Category" in df.columns:
    df["business_category1"] = df["Category"].apply(extract_primary_category)
else:
    df["business_category1"] = np.nan

# 6) Final columns for export (including identifiers)
core_cols = [
    "Stars", "Star_Bucket",
    "computed_sentiment", "Confidence",
    "Emotion", "Emotion_Confidence",
    "Review", "Date", "City", "State",
    "business_category1", "business_name",
    "Review_Len_words", "Review_Len_chars", "Review_Len",
    "ExclaimFrac", "AllCapsFrac", "NegationFlag",
    "Business_Avg_Rating", "User_Avg_Star_Rating", "Business_Is_Open",
    "user_review_count", "business_review_count",
    "business_id", "user_id", "review_id"
]
core_cols += [c for c in df.columns if c.startswith("emotion_")]
core_cols = [c for c in core_cols if c in df.columns]

df_use = df[core_cols].copy()
display(df_use.head(3))

,Stars,Star_Bucket,computed_sentiment,Confidence,Emotion,Emotion_Confidence,Review,Date,City,State,business_category1,business_name,Review_Len_words,Review_Len_chars,Review_Len,ExclaimFrac,AllCapsFrac,NegationFlag,Business_Avg_Rating,User_Avg_Star_Rating,Business_Is_Open,user_review_count,business_review_count,business_id,user_id,review_id,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_neutral,emotion_sadness,emotion_surprise
0,4.0,High,0.996471,0.996471,neutral,0.680843,We were a bit weary about trying the Shellfish...,2016-05-31 02:14:54,Santa Barbara,CA,Live/Raw Food,Santa Barbara Shellfish Company,327,1770,327,0.000000,0.003058,1,5.0,4.12,1,338,2404,SZU9c8V2GuREDN5KgyHFJw,JYYYKt6TdVA4ng9lLcXt_g,4zopEEPqfwm-c_FNpeHZYw,0.0,0.0,0.0,0.000000,0.680843,0.0,0.000000
1,3.0,Mid,0.999464,0.999464,joy,0.973149,Great experience purchasing a washer and dryer...,2017-08-07 19:51:10,Springfield,PA,Home Services,Sears Home Appliance Showroom,128,713,128,0.002805,0.000000,1,5.0,3.00,0,1,8,XTQ3iEaLMUbJg2DrcodslA,jrOeN_F9lz5415C8JLUKNg,LLzom-2TITa4gasV7_fCCA,0.0,0.0,0.0,0.973149,0.000000,0.0,0.000000
2,2.5,Low,0.996508,0.996508,surprise,0.473677,Awesome detailing job. My car was DIRTY. Hadn'...,2017-01-27 20:05:51,Tampa,FL,Auto Detailing,Mister Car Wash,22,119,22,0.000000,0.045455,0,5.0,3.75,1,4,133,UYxGgnJ4Z2lOehK23HSIlw,tcT7C-lixeyg19jbuAR0jw,CEcndvfWE_kA6gPtJOZRYw,0.0,0.0,0.0,0.000000,0.000000,0.0,0.473677


BLOCK 7 - ADDRESS SCORE IMBALNCE (FOR MODELING)

In [7]:
vc = df_use["Star_Bucket"].value_counts(dropna=False)
print("Star_Bucket counts:\n", vc, "\nProportions:\n", (vc/vc.sum()).round(3))

#Inverse-frequency class weights (Eddie can pass to sklearn)
tot = vc.sum()
class_weights = {k: float(tot/(len(vc)*v)) for k,v in vc.items()}
print("Suggested class weights:", class_weights)

# (Optional) balanced EDA sample (don’t export)
# from sklearn.utils import resample
# groups = {k: g for k,g in df_use.groupby("Star_Bucket")}
# n = min(len(g) for g in groups.values())
# df_balanced = pd.concat([resample(g, replace=False, n_samples=n, random_state=42) for g in groups.values()]) \
#                 .sample(frac=1.0, random_state=42).reset_index(drop=True)
# print("Balanced sample:", df_balanced.shape)


Star_Bucket counts:
 Star_Bucket
High    11955
Low      1582
Mid      1463
Name: count, dtype: int64 
Proportions:
 Star_Bucket
High    0.797
Low     0.105
Mid     0.098
Name: count, dtype: float64
Suggested class weights: {'High': 0.41823504809703055, 'Low': 3.1605562579013906, 'Mid': 3.417634996582365}


BLOCK 8 - LIGHT VALIDATIONS

In [8]:
required_now = ["Stars", "Star_Bucket", "computed_sentiment", "Confidence", "Review", "Date"]
missing_now = [c for c in required_now if c not in df_use.columns]
assert not missing_now, f"Missing required columns: {missing_now}"

# Ranges / sets
assert df_use["Stars"].between(1, 5).all(), "Stars out of [1,5]"
assert df_use["Confidence"].between(0, 1).all(), "Confidence out of [0,1]"
assert df_use["computed_sentiment"].between(-1, 1).all(), "computed_sentiment out of [-1,1]"
assert set(df_use["Star_Bucket"].dropna().unique()) <= {"Low", "Mid", "High"}, "Unexpected Star_Bucket values"

# Optional fields if present
if "Emotion_Confidence" in df_use.columns:
    assert df_use["Emotion_Confidence"].between(0, 1).all(), "Emotion_Confidence out of [0,1]"
if "Review_Len_words" in df_use.columns:
    assert (df_use["Review_Len_words"] >= 0).all(), "Review_Len_words has negatives"
if "Review_Len" in df_use.columns:
    assert (df_use["Review_Len"] >= 0).all(), "Review_Len has negatives"

# Date dtype check
if "Date" in df_use.columns:
    assert pd.api.types.is_datetime64_any_dtype(df["Date"]), "Date not parsed to datetime"

print("Validations passed.")

Validations passed.


BLOCK 9 - DATA DICTIONARY (FOR DOCUMENTATION RUBRIC)

In [9]:
# ----- BLOCK 9: Data Dictionary (FINAL) -----
import pandas as pd

desc = {
    "Stars": "User’s star rating (int in [1,5])",
    "Star_Bucket": "Categorized rating: Low (1–2), Mid (3), High (4–5)",
    "computed_sentiment": "Confidence-weighted sentiment (label×Confidence) clipped to [-1,1]",
    "Confidence": "Model confidence for sentiment prediction [0,1]",
    "Emotion": "Primary emotion label extracted from text",
    "Emotion_Confidence": "Model confidence score for the emotion label [0,1]",
    "Review": "Raw review text",
    "Date": "Timestamp of the review",
    "City": "City where the business is located",
    "State": "State where the business is located",
    "business_category1": "Primary business category (first listed label parsed from categories field)",
    "business_name": "Name of the business (identifier for location/category grouping)",
    "Review_Len_words": "Word count of review",
    "Review_Len_chars": "Character count of review",
    "Review_Len": "Alias of Review_Len_words for modeling compatibility",
    "ExclaimFrac": "Fraction of characters that are exclamation marks",
    "AllCapsFrac": "Fraction of tokens that are in ALL CAPS",
    "NegationFlag": "1 if review contains common negation words (e.g., 'not', 'never')",
    "Business_Avg_Rating": "Average star rating for the business",
    "User_Avg_Star_Rating": "Reviewer’s average star rating across all reviews",
    "Business_Is_Open": "Business open flag (0=closed, 1=open)",
    "user_review_count": "Total number of reviews by the user",
    "business_review_count": "Total number of reviews for the business",
    "business_id": "Unique business identifier",
    "user_id": "Unique user identifier",
    "review_id": "Unique review identifier"
}

# Add emotion_* descriptions dynamically
for c in df_use.columns:
    if c.startswith("emotion_"):
        emotion_name = c.replace("emotion_", "")
        desc[c] = f"Weighted dummy for emotion '{emotion_name}' (multiplied by Emotion_Confidence)"

# Build and export dictionary
rows = [{"column": c, "description": desc.get(c, "—")} for c in df_use.columns]
data_dictionary = pd.DataFrame(rows, columns=["column", "description"])

display(data_dictionary.head(10))
data_dictionary.to_csv("data_dictionary_df_use.csv", index=False)
print("Data dictionary saved as data_dictionary_df_use.csv")

,column,description
0,Stars,"User’s star rating (int in [1,5])"
1,Star_Bucket,"Categorized rating: Low (1–2), Mid (3), High (..."
2,computed_sentiment,Confidence-weighted sentiment (label×Confidenc...
3,Confidence,"Model confidence for sentiment prediction [0,1]"
4,Emotion,Primary emotion label extracted from text
5,Emotion_Confidence,Model confidence score for the emotion label [...
6,Review,Raw review text
7,Date,Timestamp of the review
8,City,City where the business is located
9,State,State where the business is located


✅ Data dictionary saved as data_dictionary_df_use.csv


BLOCK 10 - EXPORT CLEAN DATASET

In [10]:
out_path = "df_use.csv"
df_use.to_csv(out_path, index=False)
print(f"Exported: {out_path} | Shape: {df_use.shape}")

Exported: df_use.csv | Shape: (15000, 33)
